In [11]:
import numpy as np

dim =50    #矩阵维度

# 设置高斯分布的参数
mean = np.zeros(dim)  # dim维的均值向量
cov = np.eye(dim)     # dim维的协方差矩阵（单位矩阵）

# 生成num_samples个dim维的高斯分布样本
num_samples = 1000
data = np.random.multivariate_normal(mean, cov, num_samples)

r=0.1
C = np.zeros([dim,dim])
for i in range(dim):
    for j in range(dim):
        C[i,j]=r**abs(i-j)

data = data@(C**0.5)

data_eval = np.random.multivariate_normal(mean, cov, num_samples)@(C**0.5)
print(data.shape)

# 计算样本协方差矩阵
SCM = data_eval.T@data_eval/num_samples
print(SCM)

(1000, 50)
[[ 1.11040058  0.66562765  0.23665935 ...  0.00526888 -0.0256904
  -0.0476118 ]
 [ 0.66562765  1.19340881  0.61816087 ...  0.02506802 -0.01604705
   0.00840254]
 [ 0.23665935  0.61816087  1.15360688 ...  0.05120017  0.05253062
   0.01766275]
 ...
 [ 0.00526888  0.02506802  0.05120017 ...  1.31611301  0.718714
   0.31890449]
 [-0.0256904  -0.01604705  0.05253062 ...  0.718714    1.251185
   0.70383651]
 [-0.0476118   0.00840254  0.01766275 ...  0.31890449  0.70383651
   1.0694355 ]]


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

class TransformerModel(nn.Module):
    def __init__(self, n, m, d_model, nhead, num_encoder_layers):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(m, d_model)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead), num_encoder_layers)
        self.fc = nn.Linear(d_model, m)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=0)
        w = self.fc(x)
        w = self.softmax(w)
        return w

def compute_loss(x, w):
    r = torch.matmul(x, w)
    r_mean = r.mean()
    loss = ((r - r_mean) ** 2).mean()  # 方差
    return loss

# 数据
n = 1000  # 天数
m = 50   # 股票数量
d_model = 64
nhead = 8
num_encoder_layers = 3

x = torch.from_numpy(data).float()

# 模型
model = TransformerModel(n, m, d_model, nhead, num_encoder_layers)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 训练
epochs = 200
k = 2  # 分割数量
s = int(n / k)  #  每份数量
train_ratio = 0.8
train_losses = []
test_losses = []
for epoch in range(epochs):
    total_train_loss = 0
    total_test_loss = 0
    for i in range(k):
        # 分割数据
        train_data = x[i*s:int(i*s+s*train_ratio)]
        test_data = x[int(i*s+s*train_ratio):i*s+s]

        model.train()
        optimizer.zero_grad()
        w = model(train_data)
        train_loss = compute_loss(test_data, w)
        train_loss.backward()

        # # 梯度裁剪
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        # 计算测试损失
        model.eval()
        with torch.no_grad():
            test_loss = compute_loss(test_data, w)

        total_train_loss += train_loss.item()
        total_test_loss += test_loss.item()

    avg_train_loss = total_train_loss / k
    avg_test_loss = total_test_loss / k
    train_losses.append(avg_train_loss)
    test_losses.append(avg_test_loss)

    if epoch % 10 == 0:
        # print(f'Epoch {epoch}, Avg Train Loss: {avg_train_loss}, Avg Test Loss: {avg_test_loss}')
        print(f'Epoch {epoch}, Avg Loss: {avg_train_loss}')

# 训练结果
w2 = model(x[:80]).detach().numpy()
print("权重:", w2)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 0, Avg Loss: 0.07443331927061081
Epoch 10, Avg Loss: 0.06275120563805103
Epoch 20, Avg Loss: 0.06138725392520428
Epoch 30, Avg Loss: 0.06077142432332039
Epoch 40, Avg Loss: 0.060169585049152374
Epoch 50, Avg Loss: 0.05940084904432297
Epoch 60, Avg Loss: 0.058356428518891335
Epoch 70, Avg Loss: 0.0570683553814888
Epoch 80, Avg Loss: 0.05570976622402668
Epoch 90, Avg Loss: 0.05445376597344875
Epoch 100, Avg Loss: 0.0535771269351244
Epoch 110, Avg Loss: 0.0529395267367363
Epoch 120, Avg Loss: 0.05246035382151604
Epoch 130, Avg Loss: 0.052081283181905746
Epoch 140, Avg Loss: 0.05175739526748657
Epoch 150, Avg Loss: 0.05148409679532051
Epoch 160, Avg Loss: 0.05124539136886597
Epoch 170, Avg Loss: 0.051041822880506516
Epoch 180, Avg Loss: 0.050852783024311066
Epoch 190, Avg Loss: 0.05069912038743496
权重: [0.04081721 0.02163453 0.0073274  0.01136523 0.03702924 0.0150595
 0.04635182 0.0007922  0.0193604  0.02416335 0.0035845  0.02400509
 0.01640315 0.00216026 0.00529795 0.02045242 0.03157

In [13]:
model.eval()
with torch.no_grad():
    w_nn = model(torch.from_numpy(data_eval).float())  # 计算新的权重
print(w_nn)

tensor([0.0471, 0.0153, 0.0081, 0.0131, 0.0334, 0.0117, 0.0479, 0.0007, 0.0131,
        0.0310, 0.0027, 0.0188, 0.0112, 0.0027, 0.0050, 0.0168, 0.0247, 0.0027,
        0.0574, 0.0794, 0.0180, 0.0093, 0.0878, 0.0062, 0.0028, 0.0104, 0.0022,
        0.0037, 0.0287, 0.0103, 0.0045, 0.0063, 0.0274, 0.0026, 0.0138, 0.0029,
        0.0024, 0.0376, 0.0091, 0.0772, 0.0011, 0.0058, 0.0157, 0.0476, 0.0064,
        0.0206, 0.0164, 0.0062, 0.0322, 0.0422])


In [14]:
def risk_true(C):
  # 创建全 1 向量
  ones_vector = np.ones(50)
  return 1/np.dot(ones_vector.T, np.dot(np.linalg.inv(C), ones_vector))

def risk_nn(w):
  return np.dot(w.T, np.dot(C, w))

def risk_scm(SCM, C):
  ones_vector = np.ones(50)

  SCM_inv = np.linalg.inv(SCM)

  # 计算 Σ^(-1) 1
  inv_ones_vector = np.dot(SCM_inv, ones_vector)

  # 计算 1^T Σ^(-1) 1
  scalar = np.dot(ones_vector.T, inv_ones_vector)

  w_scm = inv_ones_vector / scalar
  return np.dot(w_scm.T, np.dot(C, w_scm))


print("risk_ture:", risk_true(C))
print("risk_nn:", risk_nn(w2))
print("risk_scm:", risk_scm(SCM, C))

risk_ture: 0.0243362831858407
risk_nn: 0.042940504833107095
risk_scm: 0.03331865776308151
